<a href="https://colab.research.google.com/github/Vicente-Reis/Projeto_Busca_Cupons_Ofertas/blob/main/C%C3%B3pia_de_Projeto_final_Alura__Busca_Cupons_Ofertas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import datetime
import tkinter as tk


def buscar_cupons_por_produto(produto):
    """
    Busca cupons para um determinado produto utilizando a API do Cuponomia e scraping.

    Args:
        produto (str): Nome do produto para o qual se deseja buscar cupons.
    """

    # Substitua "SEU_TOKEN_DE_API" por um token válido da sua API de cupons
    url_api = f"https://www.cuponomia.com.br/desconto?produto={produto}&token=SECRET_KEY"

    from google.colab import userdata
    api_key = userdata.get('SECRET_KEY')

    try:
        response = requests.get(url_api)
    except requests.exceptions.RequestException as e:
        print(f"Erro ao fazer requisição à API: {e}")
        return []  # Retorna lista vazia em caso de erro na requisição

    # Verifica se a resposta está vazia
    if not response.content:
        print("Resposta vazia recebida da API.")
        return []  # Retorna lista vazia em caso de resposta vazia

    # Verifica o tipo de conteúdo da resposta (opcional)
    # if response.headers['Content-Type'] != 'application/json':
    #    print("Resposta não está no formato JSON.")
    #    return []  # Retorna lista vazia em caso de resposta não JSON

    try:
        cupons_api = response.json()
    except JSONDecodeError as e:
        print(f"Erro ao decodificar resposta JSON: {e}")
        cupons_api = []  # Define como lista vazia ou trate o erro de forma diferente

    # Utilize scraping para buscar cupons em sites especializados
    # Exemplo de como buscar cupons no site "Cuponomia":
    url_cuponomia = f"https://www.cuponomia.com.br/desconto?q={produto}"
    response = requests.get(url_cuponomia)
    soup = BeautifulSoup(response.content, 'html.parser')

    cupons_scraping = []
    for cupom in soup.find_all('div', class_='cupom'):
        codigo_cupom = cupom.find('span', class_='codigo-cupom').text
        loja = cupom.find('span', class_='loja').text
        desconto = cupom.find('span', class_='desconto').text
        data_validade = cupom.find('span', class_='data-validade').text

        cupons_scraping.append({
            'codigo_cupom': codigo_cupom,
            'loja': loja,
            'desconto': desconto,
            'data_validade': data_validade
        })

    # Combine os cupons da API e do scraping em uma lista única
    cupons = cupons_api + cupons_scraping

    # Filtre e ordene os cupons por relevância, data de validade e confiabilidade da fonte
    cupons_filtrados = filtrar_e_ordenar_cupons(cupons)

    return cupons_filtrados


def filtrar_e_ordenar_cupons(cupons):
    """
    Filtra e ordena os cupons por data de validade e desconto.

    Args:
        cupons (list): Lista de dicionários contendo informações sobre os cupons.

    Returns:
        list: Lista de dicionários contendo os cupons filtrados e ordenados.
    """

    # Filtre cupons por data de validade (excluindo cupons expirados)
    cupons_validos = [cupom for cupom in cupons if cupom['data_validade'] >= datetime.today()]

    # Ordene os cupons por desconto (maior para menor)
    cupons_validos.sort(key=lambda cupom: float(cupom['desconto'].replace('%', '')), reverse=True)

    return cupons_validos


def exibir_cupons(cupons):
    """
    Exibe os cupons de forma clara e organizada.

    Args:
        cupons (list): Lista de dicionários contendo informações sobre os cupons.
    """

    if not cupons:
        print("Nenhum cupom")


In [ ]:
def imprimir_mensagem(mensagem):
  """
  Imprime uma mensagem na tela.

  Args:
    mensagem (str): Digite o produto da busca
  """
  print(mensagem)

imprimir_mensagem("")
